In [ ]:
%pip install datasets

In [6]:
from datasets import load_dataset
dataset = load_dataset("beomi/KoAlpaca-v1.1a")
dataset

c:\Users\SAMSUNG\miniconda3\envs\openai\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SAMSUNG\.cache\huggingface\hub\datasets--beomi--KoAlpaca-v1.1a. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 21155/21155 [00:00<00:00, 109123.50 examples/s]


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [7]:
print(dataset['train']['instruction'][1])
print(dataset['train']['output'][1])
print(dataset['train']['url'][1])

스웨터의 유래는 어디에서 시작되었나요?
스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다.
https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080102&docId=47833655


In [ ]:
# openai 파인튜니
# sft(Supervised Fine-Tuning) 포멧
# json

{
    "messages" : [
    {'role':'user', 'content' : '질문'},
    {'role':'assistant', 'content' : '정답'}]
}

In [ ]:
import json 
outpot_file = 'KoAlpaca.jsonl' # jsonl : 제이슨 형태를 리스트로 만들어 놓은것
with open(outpot_file,'w', encoding='utf-8') as f:
    for data in dataset['train'] :
        data['instruction']
        data['output']
        data = {
            "messages" : [
                {'role':'user', 'content' : data['instruction']},       # user 사용자의 질문
                {'role':'assistant', 'content' : data['output']}        # assistant 시스템이 정답을 말하는거
              ]
        }
        f.write(json.dumps(data,ensure_ascii=False) + '\n')

In [14]:
# openai에 업로드
from openai import OpenAI
client = OpenAI()
file = client.files.create(
    file = open('KoAlpaca.jsonl','rb'),
    purpose= 'fine-tune')
print(f'업로드된 파일 id : {file.id}')

업로드된 파일 id : file-9gzYiZvk8KZ1fTtLdcE76H


In [17]:
# 파인튜닝
job = client.fine_tuning.jobs.create(
    training_file = file.id,
    model= 'gpt-4o-mini-2024-07-18'
)
print(f'파인튜닝 job id = {job.id}')

파인튜닝 job id = ftjob-dLXgoKRycGfKQUslZ0vPOGsV
